In [7]:
# 1. Import Required Libraries
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import psi4

# 2. Read the SMILES data from your existing 'smiles.csv' file
smiles_df = pd.read_csv("smiles.csv")

# Check the first few rows of the dataframe to ensure it's loaded correctly
print(smiles_df.head())

# 3. Convert SMILES to RDKit Molecules and Compute Atom Counts
def add_chemical_info(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None, None
    n_atoms = mol.GetNumAtoms()
    return mol, n_atoms

smiles_df['ROMol'], smiles_df['n_atoms'] = zip(*smiles_df['smiles'].apply(add_chemical_info))

# 4. Generate 3D Coordinates for RDKit Molecules (If Not Already Present)
def generate_3d_coordinates(mol):
    if mol is None:  # Skip invalid molecules
        return None
    try:
        mol = Chem.AddHs(mol)  # Add hydrogens first (required for 3D generation)
        success = AllChem.EmbedMolecule(mol, randomSeed=42)  # Generate 3D coordinates
        if success != 0:
            print(f"Warning: 3D generation failed for {Chem.MolToSmiles(mol)}")
            return None
        AllChem.UFFOptimizeMolecule(mol)  # Optimize geometry using UFF (Universal Force Field)
        return mol
    except Exception as e:
        print(f"Error generating 3D coordinates for molecule: {e}")
        return None

# Apply 3D coordinate generation to all molecules
smiles_df['3D_mol'] = smiles_df['ROMol'].apply(generate_3d_coordinates)

# 5. Check the DataFrame with the newly added 'ROMol', 'n_atoms', and '3D_mol' columns
print(smiles_df.head())

# 6. Convert RDKit Molecule to XYZ Format
def mol_to_xyz(mol):
    if mol is None:
        return None
    try:
        mol_xyz = Chem.MolToXYZBlock(mol)  # Convert to XYZ format
        return mol_xyz
    except Exception as e:
        print(f"Error converting RDKit molecule to XYZ: {e}")
        return None

# Convert molecules to XYZ format
smiles_df['xyz'] = smiles_df['3D_mol'].apply(mol_to_xyz)

# 7. Set Up Psi4 for Energy Optimization
def compute_energy(mol_xyz):
    if mol_xyz is None:
        return None
    try:
        # Use the XYZ format with Psi4
        psi4.geometry(mol_xyz)  # Define the molecule for Psi4
        energy = psi4.energy('scf/6-31G(d)')  # Perform energy calculation (single-point)
        return energy
    except Exception as e:
        print(f"Error calculating energy: {e}")
        return None

# 8. Perform Energy Calculations and Add to DataFrame
# We will now add the energy values as a new column to the DataFrame.
smiles_df['energy'] = smiles_df['xyz'].apply(compute_energy)



        smiles
0     C1CCCCC1
1  C1=CC=CC=C1
2      CC(=O)O
3     C1COCCO1
4          CCO
        smiles                                             ROMol  n_atoms  \
0     C1CCCCC1  <rdkit.Chem.rdchem.Mol object at 0x1508481cb350>      6.0   
1  C1=CC=CC=C1  <rdkit.Chem.rdchem.Mol object at 0x1508481cb270>      6.0   
2      CC(=O)O  <rdkit.Chem.rdchem.Mol object at 0x1508481cb2e0>      4.0   
3     C1COCCO1  <rdkit.Chem.rdchem.Mol object at 0x1508481cb200>      6.0   
4          CCO  <rdkit.Chem.rdchem.Mol object at 0x1508481cb190>      3.0   

                                             3D_mol  
0  <rdkit.Chem.rdchem.Mol object at 0x1508481cb820>  
1  <rdkit.Chem.rdchem.Mol object at 0x1508481cbb30>  
2  <rdkit.Chem.rdchem.Mol object at 0x1508481cb6d0>  
3  <rdkit.Chem.rdchem.Mol object at 0x1508481cb430>  
4  <rdkit.Chem.rdchem.Mol object at 0x1508481cba50>  

Scratch directory: /tmp/

*** tstart() called on cm024.hpc.nyu.edu
*** at Thu Nov 14 15:39:58 2024

   => Loading Basis Se

[15:39:58] SMILES Parse Error: unclosed ring for input: 'CO2'


   1.007825032230
         H           -2.292307976031     0.206531726589    -0.501626988015     1.007825032230
         H           -1.895730976031     0.819031726589     1.133287011985     1.007825032230
         H           -0.499477976031    -0.979508273411     1.652575011985     1.007825032230
         H           -1.644883976031    -1.793351273411     0.536751011985     1.007825032230

  Running in c1 symmetry.

  Rotational constants: A =      0.14421  B =      0.13875  C =      0.08331 [cm^-1]
  Rotational constants: A =   4323.18096  B =   4159.73377  C =   2497.71083 [MHz]
  Nuclear repulsion =  254.387825167654455

  Charge       = 0
  Multiplicity = 1
  Electrons    = 48
  Nalpha       = 24
  Nbeta        = 24

  ==> Algorithm <==

  SCF Algorithm Type is DF.
  DIIS enabled.
  MOM disabled.
  Fractional occupation disabled.
  Guess Type is SAD.
  Energy threshold   = 1.00e-06
  Density threshold  = 1.00e-06
  Integral threshold = 1.00e-12

  ==> Primary Basis <==

  Basis S

In [8]:
# 9. Display the DataFrame with Molecules and Their Energies
print(smiles_df[['smiles', 'n_atoms', 'energy']])


        smiles  n_atoms      energy
0     C1CCCCC1      6.0 -234.191414
1  C1=CC=CC=C1      6.0 -230.701592
2      CC(=O)O      4.0 -227.782182
3     C1COCCO1      6.0 -305.819130
4          CCO      3.0 -154.070191
5           CO      2.0 -115.031280
6          CO2      NaN         NaN
7     C1CNCCO1      6.0 -285.987786
8    CCN(CC)CC      7.0 -290.360103
